In [123]:
##All imports I used for the following
import matplotlib.pyplot as plt
import numpy as np
from scipy import special
import scipy as sp
import scipy.stats
import pandas as pd
import csv
import math
#I like progressbar, because I am rather impatient
import progressbar
import copy

from scipy import stats

#this just changes the window size for the figures, it is nice for larger displays
plt.rcParams['figure.figsize'] = [12, 6]

In [2]:
##Each year is a separate file, so you must concatonate all the filenames to be read
def return_filenames():
    filenames = []
    #likely this is a different directory for whomever 
    general = 'Data/paperscape-data-master/pscp-' 
    csv = '.csv'
    #normally 1991 - 2018
    for i in range(1991, 2018):
        filenames.append(general + str(i) + csv)
    return filenames

In [3]:
##These are the dataframes that contain all the csv files
def get_df_with_data():
    filenames = return_filenames()
    nam = ['ID', 'Field', 'Found-Ref', 'Total-Ref', 'Ref-List', 'Authors', 'Title']
    #there were some difficulties with the data that all the parameters of the read_csv call deal with
    data_list = [pd.read_csv(i, sep=';', names=nam, dtype = str, comment = '#', header=None, index_col = False, quoting=3) for i in filenames]
    return data_list

# Primary approach

In [4]:
#this section details setting up the citation networks
#i.e. finding all papers
#putting them in their relevant fields
#collecting all citations any particular paper has obtained from other papers within its field

#note: I do not actually build true networks-- papers are not connected to one another
#this is because we only care about citation accrual, not the actual conectedness of the network
#although adding this might not be very difficult

In [5]:
#object representing a paper
class paper_details:
    def __init__(self, authors, ID, field, total_ref, found_ref, ref_list):
        self.authors = authors #all authors of the paper, usually a list
        self.ID = ID #unique ID
        self.field = field #the field the paper is published in
        self.total_ref = total_ref #total number of references the paper makes
        self.found_ref = found_ref #actual number of references which could actually be made-- i.e. show up in the network
        self.citations = 0 
        self.ref_list = ref_list #list of references that it makes, meaning a list of unique IDs
        self.order = 0
    def cited(self):
        self.citations += 1
    

In [6]:
##Dictionary storing papers by their unique ID
def make_data_lookup(data_list):
    pbar = progressbar.ProgressBar()
    data_lookup = {}
    for data in pbar(data_list):
        #I always assume a paper is part of the citation network related to the first field listed in the data
        a = data.Field.str.split('.', expand = True)
        a = a[0].str.split(',', expand = True)
        data['Field'] = a[0] #i.e. this
        
        for index, row in data.iterrows(): 
            if pd.isnull(row['Authors']): #no authors listed in the data
                aut = []
            else: 
                aut = row["Authors"].split(',') #get all unique authors
            
            if pd.isnull(row['Ref-List']): #if paper doesnt make citations to papers in arXiv database
                ref_list = []
            else:
                ref_list = row["Ref-List"].split(',') #all papers in arXiv database that the paper cites
            
            #intialize the paper-- make a paper object
            data_lookup[row['ID']] = paper_details(aut, str(row['ID']), row['Field'], int(row["Total-Ref"]), int(row["Found-Ref"]), ref_list)

    return data_lookup


In [7]:
##build out the citations that any paper obtains
def collect_citations(data_lookup, verbose = False):
    for key, value in data_lookup.items(): 
        if value.found_ref != 0:
            for c in value.ref_list: #iterate through IDs in the papers reference list
                    try: #check if ID is even found
                        if data_lookup[c].field == value.field: #check that it has same field 
                            data_lookup[c].cited()
                    except KeyError:
                        if verbose:
                            print('{} not found in Network'.format(c))

In [8]:
##make a list of fields--so accessing individual citation networks is possible
def all_fields(data_list):
    fields = []
    for data in data_list:
        fields.extend(data.Field)
    return list(set(fields))

In [9]:
##Assigns all papers to thier relevant field
def paper_dic(fields, data_lookup):
    dic_by_field = {}
    for field in fields:
        dic_by_field[field] = []
    for key, value in data_lookup.items(): 
        temp = key.split('/')
        value.ID = str(temp[-1])
        dic_by_field[value.field].append(value)
    return dic_by_field

In [10]:
#####################################
#Implement the functions

In [11]:
data_list = get_df_with_data()
data_lookup = make_data_lookup(data_list)

100% (27 of 27) |########################| Elapsed Time: 0:02:09 Time:  0:02:09


In [12]:
collect_citations(data_lookup)

In [13]:
fields = all_fields(data_list)

In [14]:
dic_by_field = paper_dic(fields, data_lookup)

# Secondary approach 

In [15]:
#this section also details setting up the citation networks
#i.e. finding all papers
#putting them in their relevant fields
#collecting all citations any particular paper has obtained from other papers within its field
#however, in this approach a paper can be in multiple fields
#if a paper exists in multiple fields, it is a different object in each field
#this means that the "same" paper will be different objects and have different number of citations depending on field 

#note, this approach is not necessarily recommended
#confer with Manuel




In [16]:
##Dictionary storing papers by their unique ID
def make_data_lookup_secondary(data_list):
    pbar = progressbar.ProgressBar()
    data_lookup = {}
    
    for data in pbar(data_list):
        for index, row in data.iterrows():
            #here is where the function differs from the primary approach
            #we instead store all fields the paper was classified as
            a = row["Field"].split(',')
            field_list = []
            for i in a:
                temp = i.split('.')
                field_list.append(temp[0])
            if pd.isnull(row['Authors']):
                aut = []
            else: 
                aut = row["Authors"].split(',')
            if pd.isnull(row['Ref-List']):
                ref_list = []
            else:
                ref_list = row["Ref-List"].split(',') #all papers in arXiv database that the paper cites
            data_lookup[row['ID']] = paper_details(aut, str(row['ID']), field_list, int(row["Total-Ref"]), int(row["Found-Ref"]), ref_list)


    return data_lookup

In [17]:
#seperate papers by field, i.e. build citation networks
def paper_dic_secondary(fields, data_lookup):
    dic_by_field = {}
    dic_by_field_lookup = {}
    for field in fields:
        dic_by_field[field] = []
        dic_by_field_lookup[field] = {}
    for key, value in data_lookup.items(): 
        temp = key.split('/')
        value.ID = str(temp[-1])
        for field_ in value.field:
            paper = copy.deepcopy(value) #here we create a new paper object for each field a paper is listed as
            paper.field = field_
            dic_by_field[field_].append(paper) #add each of these paper objects to the relevent field
            dic_by_field_lookup[field_][key] = paper
    return dic_by_field, dic_by_field_lookup

In [18]:
#now that we have created papers for each field
#we can give each paper object the relevant number of citations
#recall each paper gets citations from papers within its own field
def collect_citations_secondary(dic_by_field, dic_by_field_lookup):
    pbar = progressbar.ProgressBar()
    for key, value in pbar(dic_by_field.items()):
        for paper in value:
            if paper.found_ref != 0:
                for c in paper.ref_list:
                        try:
                            #this makes sure that papers only cite papers within their field
                            dic_by_field_lookup[key][c].cited() 
                            #if a paper is in the same field then cite, otherwise KeyError
                        except KeyError:
                            continue

In [19]:
# data_lookup = make_data_lookup_secondary(data_list) #remember to remake the data_list

# dic_by_field, dic_by_field_lookup = paper_dic_secondary(fields, data_lookup)

# collect_citations_secondary(dic_by_field, dic_by_field_lookup)

# Begin Analysis

In [20]:
#In this section I sort the papers by the date of publication
#This is necessary to fit the Prices model
#Since we inherently assume that index of publication is a primary factor in the number of citations a paper obtains

In [21]:
#the data changes shape mid way through 2007
#meaning the method for sorting date of publication has to change mid way through the data
def sort_by_ID(fields, dic_by_field):
    papers_grouped = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        
        papers_grouped[field] = {}
        papers_grouped[field]["90s"] = []
        papers_grouped[field]["early_00s"] = []
        papers_grouped[field]["later_00s"] = []

        for paper in dic_by_field[field]:
            #this is very tailored to the data
            
            #check the first 2 characters of the ID--they give the year
            if paper.ID[:2] in ['91', '92', '93', '94', '95', '96', '97', '98', '99']:
                #a greater paper ID is correlated to a later date, so store ID as int for comparison
                paper.ID = int(paper.ID)
                papers_grouped[field]["90s"].append(paper)
            elif paper.ID[:2] in ['00', '01', '02', '03', '04', '05', '06']:
                #I add 1 to the ID, because otherwise the ID is "0***"
                #meaning when converted to an integer comparisons with "9**" are worthless
                #despite that "0**" dates happen after "9**" dates
                temp = '1' + paper.ID
                paper.ID = int(temp)
                papers_grouped[field]["early_00s"].append(paper)
            elif paper.ID[:2] == '07':
                #data changes slightly
                #mid way through the 2007, the IDs started to contain decmals
                #had to change the method for comparison of IDs
                temp = '1' + paper.ID
                if float(temp) < 10800.0:
                    paper.ID = float(temp)
                    papers_grouped[field]["later_00s"].append(paper)
                else:
                    paper.ID = int(temp)
                    papers_grouped[field]["early_00s"].append(paper)

            else:
                temp = '1' + str(paper.ID)
                paper.ID = float(temp)
                papers_grouped[field]["later_00s"].append(paper)
                
    return papers_grouped

In [22]:
#implement
papers_grouped = sort_by_ID(fields, dic_by_field)

100% (38 of 38) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [23]:
#now sort each of the three dataframes independently, then concatonate them
def sort_by_date(papers_grouped):
    sorted_and_grouped = {}
    for field in fields:
        sorted_and_grouped[field] = []
        #sort the papers by ID, accounting for data changes
        papers_grouped[field]["90s"].sort(key = lambda paper: paper.ID)
        papers_grouped[field]["early_00s"].sort(key = lambda paper: paper.ID)
        papers_grouped[field]["later_00s"].sort(key = lambda paper: paper.ID)
        #concatonate
        sorted_and_grouped[field].extend(papers_grouped[field]["90s"])
        sorted_and_grouped[field].extend(papers_grouped[field]["early_00s"])
        sorted_and_grouped[field].extend(papers_grouped[field]["later_00s"])
    return sorted_and_grouped

In [24]:
#give each player a unique order
#for later analysis
def add_order(fields, sorted_and_grouped):
    for field in fields:
        index = 1
        for paper in sorted_and_grouped[field]:
            paper.order = index
            index+=1

In [25]:
#implement
sorted_and_grouped = sort_by_date(papers_grouped)

add_order(fields, sorted_and_grouped)

In [26]:
#function to count the moving average citation count for the jth paper
def moving_avg(x, N):
    mov_avg = []
    N = int(N/2)
    print(N)
    for i in range(len(x)):
        if N+i <= len(x) and i - N >0:
            mov_avg.append((1/N)*np.sum(x[i-N:N+i]))
        elif N/2+i > len(x):
            mov_avg.append((1/(len(x[i-N:])))*np.sum(x[i-N:]))
        else:
            mov_avg.append((1/(len(x[:N+i])))*np.sum(x[:N+i]))
    return mov_avg

In [27]:
#function to count the moving average of the predicted g(t) function
def gt_avg(y, x, N, n):
    mov_avg = []
    for i in range(len(x)):
        if N+i <= len(x):
            mov_avg.append((n/N)*np.trapz(y[i:N+i], x = x[i:N+i]))
        else:
            mov_avg.append((n/(len(x[i:])))*np.trapz(y[i:N+i], x = x[i:N+i]))
    return mov_avg
#these 2 functions were very specific for the purposes of displaying some things

In [35]:
from Simulations import theoreticalG
#here you can plot citation counts of papers based on their index of publication
#useful to demonstrate that the number of citations a paper has obtained,
#is somewhat related to its date of publication
def lil_plottin(sorted_and_grouped):
    for field in fields:
        
        n = len(sorted_and_grouped[field])
        factor = 1.0
        N = 1000
        
        
        

        ind = math.floor(n*factor)
        x = [i/n for i in range(ind)]
        
        vals = [i.citations for i in sorted_and_grouped[field][n -ind:]]
        print(field)
        #uncomment the following lines to display the graphics with "moving avgs"
        #y = gt_avg(y, x, N, n) #uncomment this line
        #vals = moving_avg(vals, N) #uncomment this line 
        plt.plot(x, vals[:ind], label = 'Citation Count for '+field)
        #y = theoreticalG(dic_of_c[field], dic_of_a[field], n, []) #uncomment to plot with g(t)
        #plt.plot(x, y[:ind], color = 'red', label = 'g(t)')
        plt.xlabel('Scaled Paper Number')
        plt.ylabel('Citations')
        plt.ylim(bottom = 0, top = max(vals)+5)
        plt.legend()
        plt.savefig('citation_count/'+field+'_cit_count')
        plt.show()

This plots

In [37]:
#if you plot this with g(t), you need to fit the parameters a and c before hand
#lil_plottin(sorted_and_grouped)

In [38]:
#collect all the fields with 1000 or more papers
def collect_important_fields(fields, sorted_and_grouped):
    important_fields = []
    for field in fields:
        if len(sorted_and_grouped[field]) >= 1000:
            important_fields.append(field )
    return important_fields
    

In [39]:
#for the rest of the analysis fields is a smaller subset of all the fields
#only fields with more thatn 1000 papers are considered
fields = collect_important_fields(fields, sorted_and_grouped)

# Author and h-index Analysis

In [41]:
#this section collects the unique authors 
#and then find the h-index for every author
#and find the h-index distribution for a given m
#m is the number of publications an author makes

In [42]:
#object representing author
class an_author:
    def __init__(self, name):
        self.name = name
        self.papers = [] #list of their papers
        self.order_list = [] #list of puplication indices
        self.order_list2 = []
        self.hIndex = 0 #authors true hIndex

In [43]:
def find_authors(fields, dic_by_field): #collect all unique authors
    pbar = progressbar.ProgressBar()
    dic_by_field_authors = {}
    all_authors = []
    for field in pbar(fields):
        dic_by_field_authors[field] = {}
        for paper in dic_by_field[field]: #iterate through the papers in the field
            all_authors.extend(paper.authors)
        for author in list(set(all_authors)): #all unique authors
            dic_by_field_authors[field][author] = an_author(author)
    return dic_by_field_authors #returns a diction of all unique authors in a field

In [44]:
#assign the papers to their relevant authors
#this function is depreciated
#use which_papers_author_for_field_specific instead
def which_papers_author(fields, dic_by_field):
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        n = len(dic_by_field[field])
        print(n)
        for paper in dic_by_field[field]:
            for aut in paper.authors:
                dic_by_field_authors[field][aut].papers.append(paper.citations)
                dic_by_field_authors[field][aut].order_list.append(paper.order)
                dic_by_field_authors[field][aut].order_list2.append([paper.order/n, paper.citations])

In [45]:
#assign the papers to their relevant authors
def which_papers_author_for_field_specific(fields, dic_by_field):
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        for paper in dic_by_field[field]:
            for aut in paper.authors:
                dic_by_field_authors[field][aut].papers.append(paper)

In [46]:
from statistics import mode
#assigns the index of publication for every paper
#authors order list is filled out with paper index {1,...,n}
def one_more_step(fields, dic_by_field_authors):
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        n = len(dic_by_field[field])
        for aut, author in dic_by_field_authors[field].items():
            temp = [paper.field for paper in author.papers]
            if temp:
                tru_field = mode(temp)
                temp2 = [paper.citations for paper in author.papers if paper.field == tru_field ]
                temp3 = [paper.order for paper in author.papers if paper.field == tru_field]
                temp4 = [paper.order/n for paper in author.papers if paper.field == tru_field]
                author.papers.clear()
                author.papers.extend(temp2)
                author.order_list.extend(temp3)
                author.order_list2.extend(temp4)

In [47]:
#calculates the h-index of every author
#creates dictionaries with h-indices for all authors for each unique m
#where m is the number of papers published by an author

#as well as creates a dictionary for frequency of publication
#i.e. list of how many papers an author publishes--for all authors
def calculate_data_hIndex(fields, dic_by_field_authors):
    dic_of_hindex = {}
    dic_of_numPapers = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        dic_of_hindex[field] = {}
        dic_of_numPapers[field] = []
        for i in range(2000):#a little gross, but we dont know all the unique number of publications by authors
            dic_of_hindex[field][i] = []
        dic_of_hindex[field]["All"] = []
        for author in dic_by_field_authors[field].keys():
            dic_by_field_authors[field][author].papers.sort(reverse = True)
            
            #calculate h-index for each author
            if len(dic_by_field_authors[field][author].papers) > 0:
                dic_of_numPapers[field].append(len(dic_by_field_authors[field][author].papers)) #gives frequency of all number of papers published by an author
                k = 0
                while dic_by_field_authors[field][author].papers[k] >= k + 1 and k < len(dic_by_field_authors[field][author].papers)-1:
                    k += 1

                dic_by_field_authors[field][author].hIndex = k #assign authors h-index
                dic_of_hindex[field][len(dic_by_field_authors[field][author].papers)].append(k) #tabulate all h-indices for m publications
                dic_of_hindex[field]["All"].append(k) #all h-indices independent of publication
                
    return dic_of_hindex, dic_of_numPapers

In [48]:
#find the inter-publication times
#returns empirical distribution for number of papers published in the interim
#between all consecutive pairings of an authors publication indices
def paper_pub_freq(fields, dic_by_field_authors):
    dist_of_publication = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        dist_of_publication[field] = []
        for key, aut in dic_by_field_authors[field].items():
            aut.order_list.sort()
            temp = list(np.diff(aut.order_list))
            if temp:
                dist_of_publication[field].extend(temp)
    return dist_of_publication

In [49]:
#calculate how often frequently authors make m publications
#find the empirical distribution for publications made by an author
def deterministic_m(fields, dic_of_numPapers):
    prob_dic ={}
    for field in fields:
        temp = list(set(dic_of_numPapers[field]))   
        prob_dic[field] = {i : dic_of_numPapers[field].count(i)/len(dic_of_numPapers[field]) for i in temp}
    return prob_dic

In [50]:
#calculates the empirical total window size of publications
#i.e. how many papers between all authors first and last papers
def paper_pub_len(fields, dic_by_field_authors):
    dist_of_publication = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        dist_of_publication[field] = []
        for key, aut in dic_by_field_authors[field].items():
            aut.order_list.sort()
            if len(aut.order_list) > 1:
                dist_of_publication[field].append(aut.order_list[-1] - aut.order_list[0])
    return dist_of_publication

In [51]:
###Implementation of author analysis

In [52]:
dic_by_field_authors = find_authors(fields, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:41 Time:  0:00:41


In [53]:
which_papers_author_for_field_specific(fields, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


In [54]:
one_more_step(fields, dic_by_field_authors)

100% (21 of 21) |########################| Elapsed Time: 0:00:12 Time:  0:00:12


In [55]:
dic_of_hindex, dic_of_numPapers = calculate_data_hIndex(fields, dic_by_field_authors)

100% (21 of 21) |########################| Elapsed Time: 0:00:05 Time:  0:00:05


In [56]:
prob_dic = deterministic_m(fields, dic_of_numPapers)

In [57]:
dist_of_publication = paper_pub_freq(fields, dic_by_field_authors)

100% (21 of 21) |########################| Elapsed Time: 0:00:36 Time:  0:00:36


In [58]:
dist_of_publication_len = paper_pub_len(fields, dic_by_field_authors)

100% (21 of 21) |########################| Elapsed Time: 0:00:02 Time:  0:00:02


In [59]:
#plots all inter-publication indices
#with fitted exponential distribution
def plot_pub_dist(fields, dist_of_publication):
    
    for field in fields:
        print(field)
        lam = 1/np.mean(dist_of_publication[field])
        rv = sp.stats.expon()
        fig, ax = plt.subplots()
        m = max(dist_of_publication[field])
        print(min(dist_of_publication[field]))
        x = np.array([i for i in range(m + 1)])
        #lam = 1/scale_
        print(lam)
        ax.plot(x, lam*rv.pdf(lam*x), 'r--', label = 'Exponential pdf with $\lambda = {0:.3e}$'.format(lam))
        ax.hist(dist_of_publication[field], bins = 'auto', density = True, label = 'arXiv Inter-publication Times')
        ax.set_ylabel('Frequency')
        ax.set_xlabel('Number of Papers Between Publications')
        if field == 'hep-ex':
            ax.set_xlim(left = 0, right = .01*m)
        else:
            ax.set_xlim(left = 0, right = .15*m)
        plt.legend()
        plt.savefig('exp_dist_waiting_time/'+field+'_exp_dist')
        plt.show()

In [60]:
#plots distribution of first and last publication window indices
def plot_pub_dist_len(fields, dist_of_publication):
    
    for field in fields:
        print(field)
        fig, ax = plt.subplots()
        m = max(dist_of_publication[field])
        print(min(dist_of_publication[field]))
        x = np.array([i for i in range(m + 1)])
   
        ax.hist(dist_of_publication[field], bins = 'auto', density = True, label = 'arXiv Publication Range')
       
        ax.set_ylabel('Frequency')
        ax.set_xlabel('Number of Papers Between Publications')

        plt.legend()

        plt.show()

This Plots

In [63]:
#plot_pub_dist(fields, dist_of_publication)

In [65]:
#plot_pub_dist_len(fields, dist_of_publication_len)   #this is the distribution for the range of publications

# Fit a and c

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [68]:
#find the average number of citations each papers makes
#this is somewhat straight forward
def find_c_for_field(fields, dic_by_field):
    pbar = progressbar.ProgressBar()
    dic_of_c = {}
    for field in pbar(fields):
        temp = []
        for paper in dic_by_field[field]:
            temp.append(paper.citations)
        dic_of_c[field] = np.mean(temp)
    return dic_of_c
   

In [69]:
dic_of_c = find_c_for_field(fields, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [70]:
#find the frequency of papers based on the number of citations
#i.e. how many papers have n citations
def cit_freq(fields, dic_by_field):
    dic_cit_freq ={}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        dic_cit_freq[field] = {}
        for paper in dic_by_field[field]:
            dic_cit_freq[field][paper.citations] = 0
        for paper in dic_by_field[field]:
            dic_cit_freq[field][paper.citations] += 1
    return dic_cit_freq

In [71]:
dic_cit_freq = cit_freq(fields, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [75]:

#was a niave attempt to fit the very non-linear in degree distribution
#some of this function is still necessary though

def pick_subset(X, y, mse_dic, dic_cit_freq, field, n, middle = True, median = False):
    if middle:
        pivot = 0.5*(max(X) + min(X))
    if median:
        pivot = np.median(X)
    rang = max(X) - min(X)
    for j in [-2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0]: #permutation of pivot
        mse_dic[field][j] = {}
        if np.abs(pivot) > np.abs(j): 
            mid = pivot + j
        else:
            continue
        for i in [(.5, .5), (.5, .1), (.1, .5), (.1, .1), (.1, .2), (.2, .1), (.2, .2), (.1, .3), (.3, .1), (.3, .2), (.2, .3), (.3, .3)]:
            y_temp = []
            X_temp = []
            left = mid - i[0]*rang
            right = mid + i[1]*rang
            for key, value in dic_cit_freq[field].items():
                if value != 0 and key !=0:
                    if np.log(key) <= right and np.log(key) >= left: 
                        y_temp = np.append(y_temp, np.log(float(value)/n))
                        X_temp = np.append(X_temp, np.log(key))                         
            if len(X_temp) > 0.05*len(X) and len(X_temp) > 2:
                reg = LinearRegression().fit(np.array([X_temp]).T, np.array([y_temp]).T)
                mse = mean_squared_error(np.array([X_temp]).T, reg.predict(np.array([y_temp]).T))
                mse_dic[field][j][i] = {'mse': mse, 'regression' : reg}

In [76]:
#regress to find a
#This is not used to the regression, but still used to find the in-degree dist
#reg_dic contains the in-degree dist
def regress_a(fields, dic_cit_freq, dic_by_field):
    reg_dic = {}
    mse_dic_median = {}
    mse_dic_middle = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields): 
        n = len(dic_by_field[field])
        y = np.array([])
        X = np.array([])
        for key, value in dic_cit_freq[field].items():
            if value != 0 and key !=0:
                y = np.append(y, np.log(float(value)/n))
                X = np.append(X, np.log(key))
        mse_dic_median[field] = {}
        mse_dic_middle[field] = {}
        
        pick_subset(X, y, mse_dic_median, dic_cit_freq, field, n, middle = False, median = True) #median
        pick_subset(X, y, mse_dic_middle, dic_cit_freq, field, n) #middle
        
        reg = LinearRegression().fit(np.array([X]).T, np.array([y]).T) #regular regression
        reg_dic[field] = {'y' : y, 'X': X, 'regression': reg}
        
    return reg_dic, mse_dic_median, mse_dic_middle
        

In [77]:
#this is used to find the in-degree distribution
reg_dic, mse_dic_median, mse_dic_middle = regress_a(fields, dic_cit_freq, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


In [78]:
#another method to attempt to fit the in degree distribution
#also unused


# def find_min_mse(fields, mse_dic, reg_dic):
#     min_mse = {}
#     for field in fields:
#         min_mse[field] = {'min_mse':100000, "regression": reg_dic[field]['regression']}
#         y = reg_dic[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
#         #y = y.flatten()
#         for key, value in mse_dic[field].items():
#             for key, value in value.items():
#                 if value['regression'].coef_ < -2.5 and value['mse'] < min_mse[field]['min_mse']:
#                     min_mse[field]['min_mse'] = value['mse']
#                     min_mse[field]['regression'] = value['regression']
#     return min_mse 


In [79]:
#another attempt to fit in degree dist
#unused


# def plot_mse_reg(fields, mse_dic, reg_dic):
#     for field in fields:
#         print(field)
#         y = reg_dic[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
#         #y = y.flatten()
#         fig, ax = plt.subplots()
#         ax.scatter(reg_dic[field]['X'], reg_dic[field]['y'])
#         for key, value in mse_dic[field].items():
#             for key, value in value.items():
#                 if value['regression'].coef_ < -2.0:
#                     y = value['regression'].predict(np.array([reg_dic[field]['X']]).T)
#                     ax.plot(reg_dic[field]['X'], y)
#         ax.plot(reg_dic[field]['X'], y, color = 'red')
        
#         plt.show()

In [80]:
#plots the in degree distribution
#unused


# def plot_reg(fields, reg_dic):
#     for field in fields:
#         print(field)
#         x = np.array([np.linspace(0, max(reg_dic[field]['X']))])
#         y = reg_dic[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
        
#         x = x.flatten()
#         #y = y.flatten()
#         print(y.shape)
        
#         fig, ax = plt.subplots()
#         ax.scatter(reg_dic[field]['X'], reg_dic[field]['y'])
#         ax.plot(reg_dic[field]['X'], y, color = 'red')
#         plt.show()

In [81]:
#function definition is misleading
#This is used to plot just the indegree distribution, no fit
def plot_min_mse(fields, reg_dic):
    for field in fields:
        print(field)
        fig, ax = plt.subplots()
        ax.scatter(reg_dic[field]['X'], reg_dic[field]['y']) 
 
        ax.set_xlabel('log(k)')
        ax.set_ylabel('log(d(k))')

        plt.savefig('indegree_dist/'+field+'power_law')

        plt.show()

In [82]:
#After fitting the data, this function returns a dictionary of fitted a's
#where the keys are fields
def find_a(fields, reg_dic, dic_of_c, tails = False):
    dic_of_a = {}
    if tails:
        for field in fields:
            dic_of_a[field] = (dic_of_c[field]*((1- reg_dic[field]['regression'].coef_) -2)).flatten().item()
    else:
        for field in fields:
            dic_of_a[field] = (-dic_of_c[field]*(reg_dic[field]['regression'].coef_ +2)).flatten().item()

    return dic_of_a
        
     

In [83]:
#This is used to pick the data for the tail distribution 1.5 std from the mean
def subset_tailored_mean_and_variance(X, y):
    mean = np.mean(X)
    std = np.std(X)
    X_sub = X[(X < mean+1.5*std) & (X>mean-1.5*std)]
    y_sub = y[(X < mean+1.5*std) & (X>mean-1.5*std)]
    return(X_sub, y_sub)

In [84]:
#this was a niave way to pick data from the tail dist, 
#unused

# def pick_subset_tailored(X, y, mse_dic, dic_cit_freq, field, n, middle = False, median = True):
#     if middle:
#         pivot = 0.5*(max(X) + min(X))
#     if median:
#         pivot = np.median(X)
#     rang = max(X) - min(X)
#     for j in [-2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0]: #permutation of pivot
#         mse_dic[field][j] = {}
#         if np.abs(pivot) > np.abs(j): 
#             mid = pivot + j
#         else:
#             continue
#         for i in [(.1, .1), (.1, .2), (.2, .1), (.2, .2), (.1, .3), (.3, .1), (.3, .2), (.2, .3), (.3, .3), (.1, .4), (.4, .1),(.4, .2),(.2, .4), (.4, .3), (.3, .4), (.4,.4)]:
#             left = mid - i[0]*rang
#             right = mid + i[1]*rang
#             temp = np.array([])
#             y_temp = np.array([])
#             X_temp= np.array([])
            
#             for key, value in dic_cit_freq[field].items():
#                 if value != 0 and key !=0:
#                     if np.log(key) <= right and np.log(key) >= left: 
#                         temp = np.append(temp, float(value)/n)
#                         X_temp = np.append(X_temp, key)
#             if len(X_temp)>0:
#                 X_temp, temp = zip(*sorted(zip(X_temp,temp), key=lambda pair: pair[0]))
            
#                 for i in range(len(X_temp)):
#                     y_temp = np.append(y_temp, np.sum(temp[i:]))

#                 y_temp = np.log(y_temp)
#                 X_temp = np.log(X_temp)

#                 if len(X_temp) > 0.5*len(X) and len(X_temp) > 2:
#                     reg = LinearRegression().fit(np.array([X_temp]).T, np.array([y_temp]).T)
#                     mse = mean_squared_error(np.array([X_temp]).T, reg.predict(np.array([y_temp]).T))
#                     mse_dic[field][j][i] = {'mse': mse, 'regression' : reg}

In [85]:
#plot for tail distribution
def plot_tails(fields, reg_dic, min_mse_tails):
    plt.rcdefaults()
    for field in fields:
        print(field)
        y = reg_dic[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
        y2 = min_mse_tails[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
        X, Y = subset_tailored_mean_and_variance(reg_dic[field]['X'], reg_dic[field]['y'])
        fig, ax = plt.subplots()
        ax.scatter(reg_dic[field]['X'], reg_dic[field]['y'], label = 'Entire Tail Distribution')
        ax.scatter(X, Y, color = 'black', label = '$\mu \pm 1.5 \sigma$')
        #ax.plot(reg_dic[field]['X'], y, color = 'red')
        ax.plot(reg_dic[field]['X'], y2, color = 'red', label = 'Linear Fit')
        ax.set_xlabel('log(k)')
        ax.set_ylabel(r'$log \left( \sum_{i\geq k} d(i)\right)$')
        #ax.set_title('{} Tail Indegree Distribution'.format(field))
        plt.legend()
        plt.tight_layout()
        plt.savefig('tail_deg/'+field+'tail_dist')
        plt.show()

In [86]:
#method for both fitting and creating the tail distribution#
def tails_reg(fields, dic_cit_freq, dic_by_field, pick_subset = False, mean_subset = True):
    reg_dic = {}
    mse_dic_median = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields): 
        n = len(dic_by_field[field])
        temp = np.array([])
        y = np.array([])
        X = np.array([])
        #here is the implementation creating the tail distribution
        for key, value in dic_cit_freq[field].items():
            if value != 0 and key !=0:
                temp = np.append(temp, float(value)/n)
                X = np.append(X, key)
        X, temp = zip(*sorted(zip(X,temp), key=lambda pair: pair[0]))
        #tail dist, meaning the sum of data greater than i
        for i in range(len(X)):
            y = np.append(y, np.sum(temp[i:]))
        #take the log 
        y = np.log(y)
        X = np.log(X)
        #now fit the data
        reg = LinearRegression().fit(np.array([X]).T, np.array([y]).T) #regular regression
        reg_dic[field] = {'y' : y, 'X': X, 'regression': reg}
        mse_dic_median[field] = {}
        if pick_subset:
            pick_subset_tailored(X, y, mse_dic_median, dic_cit_freq, field, n)
        #this is the method implemented
        #1.5 std from the mean
        if mean_subset:
            X, y = subset_tailored_mean_and_variance(X, y)
            reg = LinearRegression().fit(np.array([X]).T, np.array([y]).T) #regular regression
            mse_dic_median[field] = {'y' : y, 'X': X, 'regression': reg}
    return reg_dic, mse_dic_median
    

In [87]:
#picking up subsets of the data

In [88]:
#find and fit the tail distribution
reg_dic_tails, mean_std_tails = tails_reg(fields, dic_cit_freq, dic_by_field)

100% (21 of 21) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [89]:
#this was previously used to find the fit that minimized the mse of the fit
#unused
def find_min_mse_tailored(fields, mse_dic, reg_dic):
    min_mse = {}
    for field in fields:
        min_mse[field] = {'min_mse':100000, "regression": reg_dic[field]['regression']}
        y = reg_dic[field]['regression'].predict(np.array([reg_dic[field]['X']]).T)
        #y = y.flatten()
        for key, value in mse_dic[field].items():
            for key, value in value.items():
                if value['mse'] < min_mse[field]['min_mse']:
                    min_mse[field]['min_mse'] = value['mse']
                    min_mse[field]['regression'] = value['regression']
    return min_mse 


These Plot

In [91]:
#plot_tails(fields, reg_dic_tails, mean_std_tails)

In [93]:
#plot_min_mse(fields, reg_dic)

In [94]:
#set a to be the a from the 1.5std from mean fit of tail dist
dic_of_a_from_tails_mean_std = find_a(fields, mean_std_tails, dic_of_c, tails = True)

In [95]:
#this is the method to calculate a from tail distribution
def calc_a(reg, c):
    a = (-c*(reg.coef_ + 2)).flatten().item()
    return a

In [96]:
# this was another niave implentation to attempt to fit a
#unused


# def bootstrap_a(fields, dic_by_field, dic_cit_freq, dic_of_c, N=1000):

#     dic_of_a_boot = {}
#     pbar = progressbar.ProgressBar()
#     for field in pbar(fields): 
#         n = len(dic_by_field[field])
#         y = np.array([])
#         X = np.array([])
#         #x_y_pairs = np.array([[0, 0]])
#         for key, value in dic_cit_freq[field].items():
#             if value != 0 and key !=0:
#                 y = np.append(y, np.log(float(value)/n))
#                 X = np.append(X, np.log(key))
#                 #x_y_pairs = np.concatenate((x_y_pairs, np.array([[np.log(key), np.log(float(value)/n)]])), axis = 0)
#         reg = LinearRegression().fit(np.array([X]).T, np.array([y]).T)
#         dic_of_a_boot[field] = 2*calc_a(reg, dic_of_c[field])
#         a_hat = 0
#         for i in range(N):
#             resample = np.random.choice(len(X), len(X))
#             reg = LinearRegression().fit(np.array([X[resample]]).T, np.array([y[resample]]).T)
#             a_hat += calc_a(reg, dic_of_c[field])
#         dic_of_a_boot[field] += (-a_hat/N)
#     return dic_of_a_boot
        

# Theoretical and comparison

In [97]:
#imports from the other document I wrote
from Simulations import p_k_Beta, hDist_fromBeta_equals_k, g_inverse

In [124]:
#used to find the total h-index probability distribution and the h-index distribution for a given m
#where m is number of papers an author has
#the distribution of m is empirical
#dist_dic is the total h-index distribtuion
#conditional_h is the h-index distribution for all possible h-index values given a particular m
def hDist(data, data2, fields, prob_dic, c, a):
    dist_dic = {}
    pbar = progressbar.ProgressBar()
    conditional_h = {}
    for field in pbar(fields):
        dist_dic[field] = []
        conditional_h[field] = []
        temp = sorted(list(set(data[field])))
        temp2 = sorted(list(set(data2[field]["All"])))
        for k in temp2:#loop over all possible h-index values in the data
            sum_over_m = 0
            temp3 = []
            for m in temp:#now all the possible number of papers an author could have published
                h = hDist_fromBeta_equals_k(len(data[field]), k, m, c[field], a[field])
                if h < 1e-9 or np.isnan(h):#this is to deal with machine error and problems from really small numbers
                    h = 0.0
                temp3.append(h)
                sum_over_m += h*prob_dic[field][m]
            conditional_h[field].append(temp3)
            dist_dic[field].append(sum_over_m)
            
    return dist_dic, conditional_h

In [125]:
#set dic_of_a to be the dictionary from the values fit earlier
dic_of_a = dic_of_a_from_tails_mean_std

In [126]:
hindex_dist_theoretical, conditional_h = hDist(dic_of_numPapers, dic_of_hindex, fields, prob_dic, dic_of_c, dic_of_a)

100% (21 of 21) |########################| Elapsed Time: 0:00:23 Time:  0:00:23


In [127]:
#this function is for the error bars when plotting the h-index distribution for all possible m
def suplement(field, prob_dic, conditional_h, dic_of_hindex, dic_of_numPapers):
    err_bars = []
    l_inverse = 1/len(list(set(dic_of_numPapers[field])))
    X = np.array(sorted([[item, key] for key, item in prob_dic[field].items()], key = lambda x: x[1]))[:,0]
    V = np.array([[X[i]*(1-X[i]) if j == i else  -X[i]*X[j] for j in range(len(X))] for i in range(len(X))])
    for k in range(len(list(set(dic_of_hindex[field]["All"])))):
        a_k = np.array(conditional_h[field][k])
        left, right = sp.stats.norm.interval(0.95)
        err_bars.append([np.dot(a_k, X)- left*np.sqrt(l_inverse*np.dot(np.dot(a_k, V), a_k.T)) ,np.dot(a_k, X)+right*np.sqrt(l_inverse*np.dot(np.dot(a_k, V), a_k.T))])
    return err_bars  

In [128]:
#plot h-index distribution of all possible m
def plot_for_all_m(dic_of_hindex, hindex_dist_theoretical):
    y = {}
    N_dic = {}
    for field in fields:
        N = len(dic_of_hindex[field]["All"]) #number of authors in the field
        y[field] = np.multiply(N,hindex_dist_theoretical[field]) 
        N_dic[field] = N
    for field in fields:
        print(field)
        fig, ax = plt.subplots()
        x = [i for i in range(len(hindex_dist_theoretical[field]))]
        bins_ = [i for i in range(max(list(set(dic_of_hindex[field]['All'])))+1)]
        ax.plot(x, hindex_dist_theoretical[field], 'o', color= 'red', label = 'Predicted h-index')
        e_bars = np.array(suplement(field, prob_dic, conditional_h, dic_of_hindex, dic_of_numPapers))
        ax.errorbar(x, hindex_dist_theoretical[field], yerr = e_bars.T, fmt='none', color = 'red', elinewidth = 1, label = '95% Confidence Interval')
        ax.hist(dic_of_hindex[field]["All"], bins=bins_, label = 'arXiv Found h-index', align = 'left', density = True)
        #ax.set_title('h-index Distribution for {}'.format(field))
        ax.set_ylabel('$P(h-index = k)$')
        ax.set_xlabel('h-index')
        ax.set_xticks(bins_)
        ax.set_ylim(bottom = 0)
        ax.set_xlim(left = -0.5, right = min(.4*len(bins_), 20))
        plt.legend()
        plt.savefig('all_m/'+field+'all_m_hindex')
        plt.show()

This plots

In [104]:
#plot distributions for all m
#plot_for_all_m(dic_of_hindex, hindex_dist_theoretical)

In [106]:
from Simulations import p_k_Beta, hDist_fromBeta, g_inverse
#another function to find the probability distribution for the h-index given an m
#this function is also used to select some of the possible m we want to display across all
#fields, this was because we wanted to be able to compare across fields
#returns a dictionary with h-index distribution of all the m's that are selected
#all the m's
#and a dictionary of expected values for the chosen m's
def p_k_given_m(dic_of_hindex, fields, c, a):
    dic_of_dist_for_m = {}
    pbar = progressbar.ProgressBar()
    m_all_fields = []
    ev_dic = {}
    samp_all_fields = {}
    index = 0
    for field in fields:
        #find commonality among m's for different fields
        all_m = list(dic_of_hindex[field].keys())
        if index == 0:
            m_all_fields.extend(all_m[:-1])
        else:
            m_all_fields = list(set(m_all_fields) & set(all_m[:-1]))
        index+=1
        samp_all_fields[field] = []
    samp = m_all_fields 
    for field in pbar(fields):
        dic_of_dist_for_m[field] = {}
        ev_dic[field] = {}
        temp = []
        for i in samp:
            if len(dic_of_hindex[field][i]) > 50: #make sure the sample size is large enough
                    temp.append(i)
        samp_all_fields[field].extend(temp) 
        if temp:
            for m in temp:
                dic_of_dist_for_m[field][m] = hDist_fromBeta(100, int(m), c[field], a[field])
                ev_dic[field][m] = 0
                for k in range(1, m+1): #expected h-index also
                    ev_dic[field][m] += p_k_Beta(g_inverse(k, c[field], a[field]), k, m)
    return dic_of_dist_for_m, samp_all_fields, ev_dic

In [107]:
m_dist,samp,ev_dic = p_k_given_m(dic_of_hindex, fields, dic_of_c, dic_of_a)

100% (21 of 21) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


In [108]:
#plot h-index distribution for all the chosen m--these m are chosen to be similar among fields
#for analysis purposesd
def plot_for_given_m(fields, m_dist, dic_of_hindex, ev_dic=ev_dic, fake=False, num = 100000, sample = samp, mean = True):
    for field in fields:
        if fake:
            temp = sample[fiemld]
        else:
            temp = list(m_dist[field].keys())
        for m in temp:
            if fake:
                N = int(num*prob_dic[field][m])
            else:
                N = len(dic_of_hindex[field][m])
            y =  m_dist[field][m]
            x = [i for i in range(0, m+1)]
            fig, ax = plt.subplots()
            print(N)
            print(field)
            print(m)
            bins_ = [i for i in range(max(list(set(dic_of_hindex[field][m])))+1)]
            if fake:
                ax.hist(dic_of_hindex[field][m], bins=bins_, label = 'arXiv Synthetic h-indices' , density = True, align = 'left')
            else:
                ax.hist(dic_of_hindex[field][m], bins=bins_, label = 'arXiv Found h-indices' , density = True, align = 'left')
            ax.plot(x[:len(bins_)+1], y[:len(bins_)+1], 'o',  markersize=8 ,color = 'red', label = 'Predicted h-index')
            if mean:
                #also includes the mean plotted as vertical line
                y_ev = np.linspace(0, max(y), num = 10)
                x_ev = np.array([ev_dic[field][m] for i in range(10)])
                x_avg = np.array([np.mean(dic_of_hindex[field][m]) for i in range(10)])
                ax.plot(x_ev, y_ev, color = 'red', label = 'Expected h-index')
                ax.plot(x_avg, y_ev, color = 'blue', label = 'Average h-index')
#extra legend titles
#             if fake:
#                 ax.set_title('{} Synthetic Authors all with m = {} Papers'.format(N, m), fontsize = 10)
#             else:
#                 ax.set_title('{} Authors all with m = {} Papers'.format(N, m), fontsize = 10)
            ax.set_ylabel('$P(h-index = k | m = {})$'.format(m))
            ax.set_xlabel('h-index')
            ax.set_xticks(bins_)
            plt.legend(title = '{} Authors'.format(N))
            if fake:
                plt.savefig('fake_h-index_given_m/'+str(m)+field+'h-index')
            else:
                plt.savefig('h-index_given_m/'+str(m)+field+'h-index')
            plt.show()

This plots

In [110]:
#plot_for_given_m(fields, m_dist, dic_of_hindex)

# Proof Plots

In [111]:
from Simulations import p_k_Beta, g_inverse
def p_k(c, a, m, k): #find probability h-index = k
    return p_k_Beta(g_inverse(k, c, a), k, m)

In [112]:
#scatter plots of expected h-index and true average h-index for the same m
#ev is one axis and avg is other axis
def EV_vs_avgH(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a):
    for field in fields:
        all_m = list(set(dic_of_numPapers[field]))
        avg_h = []
        EV = []
        for m in all_m:
            temp = 0
            for k in range(1, m+1):
                temp += p_k(dic_of_c[field], dic_of_a[field], m, k)
            EV.append(temp)
            avg_h.append(np.mean(dic_of_hindex[field][m]))
        print(field)
        plt.scatter(avg_h, EV)
        lim1 = max(avg_h)
        lim2 = max(EV)
        plt.xlim(0, max(lim1, lim2))
        plt.ylim(0, max(lim1, lim2))
        plt.gca().set_aspect('equal', adjustable='box')
        plt.show()

In [115]:
#EV_vs_avgH(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a)

In [116]:
#simple plots of expected h-index and average h-index
#m is on the x axis
def plot_for_m(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a):
    for field in fields:
        all_m = list(set(dic_of_numPapers[field]))
        avg_h = []
        EV = []
        for m in all_m:
            temp = 0
            for k in range(1, m+1):
                temp += p_k(dic_of_c[field], dic_of_a[field], m, k)
            EV.append(temp)
            avg_h.append(np.mean(dic_of_hindex[field][m]))
        print(field)
        
        lim1 = max(avg_h)
        lim2 = max(EV)
        fig, ax = plt.subplots()
        ax.scatter(all_m, EV)
        ax.scatter(all_m, avg_h)
        plt.show()

In [118]:
#plot_for_m(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a)

In [119]:
#supplements for the expected h-index and avg h-index function that follows
def another_suplement(m_list, m, field, dic_of_numPapers, cut_off):
    tracker = 0
    temp = []
    for m, index in zip(sorted(m_list), range(1,len(m_list)+1)):
        if dic_of_numPapers[field].count(m) > cut_off:
            tracker += 1
            
        if tracker/index >= 0.95:
            temp.append(m)
    
    return max(temp)

In [120]:
def another_suplement2(m_list, m, field, dic_of_numPapers, cut_off):
    tracker = 0
    temp = []
    for m, index in zip(sorted(m_list, reverse = True), range(1,len(m_list)+1)):
        if dic_of_numPapers[field].count(m) < cut_off:
            tracker += 1 
        if tracker/index >= 0.95:
            temp.append(m)
    return min(temp)

In [131]:
#constructs the confidence intervals for the avg h-index predicting expected value
#as well as the cutoffs that define regimes with "large" and "small" m
def confidence_int(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a):
    dic_of_ev_and_avg = {}
    for field in fields:
        
        all_m = list(set(dic_of_numPapers[field]))
        avg_h = []
        EV = []
        e_bars = []
        m_for_err = []
        avg_for_err = []
        for m in all_m:
            temp = 0
            for k in range(1, m+1):
                temp += p_k(dic_of_c[field], dic_of_a[field], m, k)
            EV.append(temp)
            
            dof = len(dic_of_hindex[field][m])-1
            if dof > 0:
                sample_mean = np.mean(dic_of_hindex[field][m])
                sample_var = stats.sem(dic_of_hindex[field][m])
                left, right = stats.t.interval(0.95, dof, loc=sample_mean, scale=sample_var)
                
                e_bars.append([(right - sample_mean) , (sample_mean - left)])
                m_for_err.append(m)
                avg_for_err.append(sample_mean)
            avg_h.append(sample_mean)
        
        cut_off1 = 25
        cut_off2 = 10
        result1 = another_suplement(all_m, m, field, dic_of_numPapers, cut_off1)
        result2 = another_suplement2(all_m, m, field, dic_of_numPapers, cut_off2)
        
        e_bars = np.array(e_bars).T
        dic_of_ev_and_avg[field] = {'m': all_m, 'ev': EV, 'avg': avg_h}
        print(field)
        lim1 = max(avg_h)
        lim2 = max(EV)
        fig, ax = plt.subplots()
        
        
        ax.set_xlim(0, len(all_m))
        ax.set_ylim(0, (max(lim1, lim2)+10))
        #ms=10, mew=1
        
        ax.scatter(all_m, EV, color = 'red', label = 'Theoretical Expected h-index')
        ax.scatter(all_m, avg_h, label ='arXiv Sample Average h-index')
        ax.errorbar(m_for_err, avg_for_err, yerr = e_bars, fmt='none', linewidth = 0.5, label='95% Confidence Interval')

        y = [i for i in range(int(max(lim1, lim2)+10))]
        
        x1 = [result1 for i in range(int(max(lim1, lim2)+10))]
        x2 = [result2 for i in range(int(max(lim1, lim2)+10))]
        ax.plot(x1, y, label = '95% of $m < {}$ have more than {} Authors'.format(result1, cut_off1))
        ax.plot(x2, y, label = '95% of $m > {}$ have less than {} Authors'.format(result2, cut_off2))
        ax.set_ylabel('h-index')
        ax.set_xlabel('Number of Papers Published $(m)$')
        ax.legend()
        plt.savefig('e_val/'+ field + '_e_val')
        #ax.set_aspect('equal', adjustable='box')
        plt.show()
    return dic_of_ev_and_avg
    

This plots

In [132]:
# plt.rcParams['figure.figsize'] = [12, 6]
#dic_of_ev_and_avg = confidence_int(fields, dic_of_numPapers, dic_of_hindex, dic_of_c, dic_of_a)

In [133]:
#this is not used
def error_for_m(fields, dic_of_ev_and_avg):
    for field in fields:
        print(field)
        plt.plot(dic_of_ev_and_avg[field]['m'], np.abs(np.array(dic_of_ev_and_avg[field]['ev']) - np.array(dic_of_ev_and_avg[field]['avg'])))
        plt.show()

In [134]:
#error_for_m(fields, dic_of_ev_and_avg)

# Some further analysis using random distribution, rather than authors

In [135]:
#create the "synthetic" authors and find the h-index distribution for these synthetic authors
#function takes a fair amount of time to run for all the fields
#you can choose the total number of authors 
#still use the proportional number of true m in every field
def select_random_papers(fields, dic_by_field, dic_of_numPapers, prob_dic, total_num_authors, samp, verb = False):
    dic_by_field_fake_h = {}
    
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        dic_by_field_fake_h[field] = {}
        j = 0
        temp = samp[field]
        paper_list = [paper.citations for paper in dic_by_field[field]]
        for m in temp:
            dic_by_field_fake_h[field][m] = []
            if verb:
                num_authors = int(len(dic_of_numPapers[field])*prob_dic[field][m])
            else:
                num_authors = int(total_num_authors*prob_dic[field][m])
            j += 1
            index = 1
            for i in range(num_authors):
                p_list = np.random.choice(paper_list, m)#sample
                k = 0
                p_list = sorted(p_list, reverse = True)
                while p_list[k] >= k + 1 and k < len(p_list)-1:
                    k += 1
                dic_by_field_fake_h[field][m].append(k)

    return dic_by_field_fake_h

In [136]:
#a small subset of fields if you want to use
#small_sub_field = ['hep-th', 'hep-ex', 'hep-ph']

In [138]:
#takes a few hours
#dic_by_field_fake_h = select_random_papers(fields, dic_by_field, dic_of_numPapers, prob_dic, 100000, samp, verb = True)

This plots

In [140]:
#plot_for_given_m(fields, m_dist, dic_by_field_fake_h, fake = True, mean = False)

# Some modifications to our distribution

In [141]:
def p_k_mod(c, a, m, x, k): #find pk
    return p_k_Beta(x, k, m)

In [142]:
def g(t, c, a): #need g(t) too
    return a*(np.power(t, (-c/(c+a)))-1)

In [143]:
#this function creates new empirically inspired distribution for every author in the data base
#it is based on their windowsize--meaning the window of indices of publications 
#then sample 100 times from this new empirical distribution
#returns a h-index distribution
def modified_dist_draw2(c, a, m, U_one, U_m, correct = True):
    dist = [0.0]*(m+1)
    if U_m != U_one:
        h = [k for k in range(0, m+1)]
        fac = 1/(U_m - U_one)
        g_inv_list = [fac*(g_inverse(k, c, a)-U_one) for k in range(0, m+1)]
        if correct:
            correction = 0.5
            g_inv_list1 = [fac*(g_inverse(k - correction, c, a)-U_one) for k in range(0, m+1)]
            g_inv_list2 = [fac*(g_inverse(k + correction, c, a)-U_one) for k in range(0, m+1)]
        else:
            correction = 0.0
        if g(U_one, c, a) < 1 - correction:
            dist[0] = 1.0
        elif g(U_m, c, a) >= m + correction:
            dist[-1] = 1.0
        else:
            for k in h[1:-1]:
                if k == 1:
                    if correct:
                        dist[k] = 1 -p_k_mod(c, a, m-2, g_inv_list1[k+1], k)
                    else:
                        dist[k] = 1 -p_k_mod(c, a, m-2, g_inv_list[k+1], k)

                elif k == m-1:
                    if correct:
                        dist[k] = p_k_mod(c, a, m-2, g_inv_list2[k+1], k)
                    else:
                        dist[k] = p_k_mod(c, a, m-2, g_inv_list[k+1], k)
                else:
                    dist[k] = p_k_mod(c, a, m-2, g_inv_list[k+1], k) - p_k_mod(c, a, m-2, g_inv_list[k+2], k+1)
        dist = np.nan_to_num(dist)
        sum_ = np.sum(dist)
        if sum_ < 1:
            dist[0] = 1-sum_ #left Over mass
        return np.random.choice(h, 100, p=dist)
    else:
        return np.array(dist)

In [144]:
#this is the driver for finding the h-index distribution with our window size modification
def driver(dic_of_c, dic_of_a, dic_of_numPapers, dic_by_field_authors, dic_by_field):
    mod_hindex = {}
    pbar = progressbar.ProgressBar()
    for field in pbar(fields):
        n = len(dic_by_field[field])
        all_m = list(set(dic_of_numPapers[field]))
        mod_hindex[field] = {}
        for m in all_m:
            mod_hindex[field][m] = []
        for key,author in dic_by_field_authors[field].items():
            if author.order_list2:
                temp = sorted(author.order_list2)
                m = len(author.order_list2)
                if m > 1:
                    mod_hindex[field][m].extend(list(modified_dist_draw2(dic_of_c[field], dic_of_a[field], m, temp[0], temp[-1], correct = False)))
                elif m == 1:
                    if g(temp[0], dic_of_c[field], dic_of_a[field]) >= 1:
                        mod_hindex[field][m].extend([1]*100)
                    else:
                        mod_hindex[field][m].extend([0]*100)   
                else:
                    print('something is wrong')
                    print(author.order_list2)
    
    return mod_hindex

In [ ]:
mod_hindex = driver(dic_of_c, dic_of_a, dic_of_numPapers, dic_by_field_authors, dic_by_field)

 66% (14 of 21) |################        | Elapsed Time: 0:03:41 ETA:   0:00:19

In [ ]:
#plots the two histograms for our new modified distribution and the true arXiv citation networks
def plot_for_given_m_two_histo(fields, m_dist1, m_dist2, sample = samp):
    for field in fields:
        temp = sample[field]
        for m in temp:
            fig, ax = plt.subplots()
            print(field)
            print(m)
            bins_ = [i for i in range(max(list(set(dic_of_hindex[field][m])))+1)]
            ax.hist((m_dist1[field][m], m_dist2[field][m]), bins=bins_, label = ['true', 'predicted'] , density = True, align = 'left')
            ax.set_ylabel('$P(h-index = k | m = {})$'.format(m))
            ax.set_xlabel('h-index')
            ax.set_xticks(bins_)
            plt.legend()
            plt.savefig('empirical_dist_continuity/'+ field +'_'+str(m))
            plt.show()

In [287]:
#plot_for_given_m_two_histo(fields, dic_of_hindex, mod_hindex)